# Dissertation

## Research into the techniques and methods to achieve state of the art accuracy in flower species identification


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
###imports###
import numpy as np
import tensorflow as tf
import scipy.io
from scipy import io
import tensorflow_hub as hub
import time
import sys
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.optimizer_v2.adam import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing import image
from tensorflow.python.framework import graph_util
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import datasets, layers, models, losses, Model
import keras
import pickle
import datetime
import time
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import OneHotEncoder
import logging
#import skimage.io
import random
import PIL
from pathlib import Path
from keras.models import Model

In [3]:
print(tf.__version__)

2.5.0


In [4]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [5]:
if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [6]:
#makes all outputs be in float format rather than exponentials
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [7]:
#Set Parameters
IMG_SIZE = 224 #this parameter sets image dimensions as 50*50
DATE = datetime.datetime.now().strftime('%d-%b-%Y')
MODEL_PATH = f'models/{DATE}/'
MODEL_NAME = 'TransferResNet152V2.model'.format(int(time.time()))
TENSORBOARD = TensorBoard(log_dir=f'logs\\{MODEL_NAME}')
log_dir=f'logs\\{MODEL_NAME}'

In [8]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [9]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Load Directory

In [10]:
URL_dir = 'E:/Github/thesis/flowerDataset/Flower'
labels_dir = 'E:/Github/thesis/flowerDataset/imagelabels.mat'
dataSplit_dir = 'E:/Github/thesis/flowerDataset/setid.mat'
loaded_images_dir = 'E:/Dissertation/loaded_images.npy'
image_train_dir ='E:/Dissertation/image_train.npy'
label_train_dir = 'E:/Dissertation/label_train.npy'
image_test_dir = 'E:/Dissertation/image_test.npy'
label_test_dir = 'E:/Dissertation/label_test.npy'
image_val_dir = 'E:/Dissertation/image_val.npy'
label_val_dir = 'E:/Dissertation/label_val.npy'

In [11]:
data_labels = io.loadmat(labels_dir)
data_labels = data_labels.items()
data_labels = list(data_labels)

In [12]:
df = pd.DataFrame(data_labels)

In [13]:
labels = df[1][3][0] #loads the labels that is stored in dataframe format

In [14]:
len(labels)

8189

In [15]:
#Got loading files code from link below
#https://stackoverflow.com/questions/30230592/loading-all-images-using-imread-from-a-given-folder
#Function gets all the images loaded up
def load_images_from_folder(URL):
    images = []
    for filename in os.listdir(URL):
        #img = cv2.imread(os.path.join(URL,filename))
        img = cv2.resize(cv2.imread(os.path.join(URL,filename), cv2.COLOR_BGR2RGB), (IMG_SIZE, IMG_SIZE))
        img = np.reshape(img,[IMG_SIZE,IMG_SIZE,3])
        if img is not None:
            images.append(img)
    np.save(loaded_images_dir, images)
#################################    return images#######################################################################

In [16]:
#No need to run this cell if image already loaded.
load_images_from_folder(URL_dir)

In [17]:
image_data = np.load(loaded_images_dir, allow_pickle=True)

In [18]:
image_data.shape

(8189, 224, 224, 3)

In [19]:
def randomised_dataset(image_data, labels):
    np.random.seed(8)
    indices = np.arange(image_data.shape[0])
    print(indices)
    np.random.shuffle(indices)
    image_data = image_data[indices]
    labels = labels[indices]
    return image_data, labels

In [20]:
image_data, labels = randomised_dataset(image_data, labels)

[   0    1    2 ... 8186 8187 8188]


In [21]:
# # Label Encoder

mlb = LabelBinarizer()
converted_labels = np.array(mlb.fit_transform(labels))

In [22]:
converted_labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [23]:
#citation code from https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
#splitting the image dataset into the ratio for training, validation and testing data
def split_data(image_data, converted_labels):
    
    train_ratio = 0.75
    validation_ratio = 0.15
    test_ratio = 0.10

    image_train, image_test, label_train, label_test = train_test_split(image_data, converted_labels, test_size=1 - train_ratio, random_state=42)

    #the test from previous line which is 25% of dataset is passed into the line below to be 
    #further split into 15% for validation and 10% for testing

    image_val, image_test, label_val, label_test = train_test_split(image_test, label_test, test_size=test_ratio/(test_ratio + validation_ratio),random_state=42) 

    print('image_train',image_train.shape)
    print('label_train',label_train.shape)
    print('image_test',image_test.shape)
    print('label_test',label_test.shape)
    print('image_val', image_val.shape)
    print('label_val', label_val.shape)

    np.save(image_train_dir, image_train)
    np.save(label_train_dir, label_train)
    np.save(image_test_dir, image_test)
    np.save(label_test_dir, label_test)
    np.save(image_val_dir, image_val)
    np.save(label_val_dir, label_val)



In [24]:
split_data(image_data,converted_labels)

image_train (6141, 224, 224, 3)
label_train (6141, 102)
image_test (820, 224, 224, 3)
label_test (820, 102)
image_val (1228, 224, 224, 3)
label_val (1228, 102)


In [25]:
image_train = np.load(image_train_dir, allow_pickle=True)
image_test = np.load(image_test_dir, allow_pickle=True)
label_train = np.load(label_train_dir, allow_pickle=True)
label_test = np.load(label_test_dir, allow_pickle=True)
image_val = np.load(image_val_dir, allow_pickle=True)
label_val = np.load(label_val_dir, allow_pickle=True)

In [26]:
image_train.shape

(6141, 224, 224, 3)

### Feature Scaling

In [27]:
# #converts the images to the range of 0 - 1.0 
image_train = image_train/255.0
image_test = image_test/255.0
image_val = image_val/255.0

In [28]:
image_train.shape

(6141, 224, 224, 3)

In [29]:
label_train.shape

(6141, 102)

In [30]:
label_test.shape

(820, 102)

In [31]:
len(image_test)

820

## Find unique labels

In [32]:
len(label_train)

6141

In [33]:
# number of classes/ flowers
NUM_CLASSES = 102

# Transfer Learning --ResNet152V2

In [34]:
from tensorflow.keras.applications  import ResNet152V2

In [35]:
resnet = ResNet152V2(
    input_shape=(224,224,3),
    weights="imagenet",
    include_top=False, # removes last layer of the resnet model
)

In [36]:
for layer in resnet.layers: #don't train existing weights
    layer.trainable = False

In [37]:
x = layers.Flatten()(resnet.output)
x = layers.Dense(1000, activation='relu')(x)
prediction = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=resnet.input, outputs=prediction)

In [38]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Total params: 158,786,750
Trainable params: 100,455,102
Non-trainable params: 58,331,648
__________________________________________________________________________________________________


In [39]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [40]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [41]:
batch_size = 64
epochs = 100
verbose= 2

In [42]:
EXPERIMENT_NAME = "transfer-learning-tests"
RUN_NAME = "resnet152v2_Dense2"

mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.set_tracking_uri('file:///E:/GoogleSync/Masters/Dissertation/MLflow/mlruns')

with mlflow.start_run(run_name=RUN_NAME) as run:
        
        mlflow.tensorflow.autolog()
        
        history = model.fit(image_train, label_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(image_val, label_val), callbacks=[tensorboard_callback])

        score = model.evaluate(image_test, label_test, batch_size=batch_size, verbose = verbose)
        
        
        
    
        
        mlflow.log_metric("test loss", score[0])
        mlflow.log_metric("test accuracy", score[1])
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)
        

2021/08/22 11:04:00 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


Epoch 1/100
96/96 - 36s - loss: 5.6126 - accuracy: 0.4809 - val_loss: 1.5935 - val_accuracy: 0.6523
Epoch 2/100
96/96 - 17s - loss: 0.1823 - accuracy: 0.9479 - val_loss: 1.2610 - val_accuracy: 0.7353
Epoch 3/100
96/96 - 17s - loss: 0.0469 - accuracy: 0.9907 - val_loss: 1.3310 - val_accuracy: 0.7419
Epoch 4/100
96/96 - 17s - loss: 0.0312 - accuracy: 0.9920 - val_loss: 1.4959 - val_accuracy: 0.7174
Epoch 5/100
96/96 - 17s - loss: 0.0118 - accuracy: 0.9982 - val_loss: 1.3035 - val_accuracy: 0.7508
Epoch 6/100
96/96 - 17s - loss: 0.0040 - accuracy: 0.9998 - val_loss: 1.2697 - val_accuracy: 0.7557
Epoch 7/100
96/96 - 17s - loss: 0.0013 - accuracy: 0.9997 - val_loss: 1.2367 - val_accuracy: 0.7598
Epoch 8/100
96/96 - 17s - loss: 6.5748e-04 - accuracy: 0.9998 - val_loss: 1.2323 - val_accuracy: 0.7614
Epoch 9/100
96/96 - 17s - loss: 7.0605e-04 - accuracy: 0.9997 - val_loss: 1.2347 - val_accuracy: 0.7598
Epoch 10/100
96/96 - 17s - loss: 3.1026e-04 - accuracy: 1.0000 - val_loss: 1.2341 - val_accu

96/96 - 17s - loss: 0.0038 - accuracy: 0.9992 - val_loss: 2.8483 - val_accuracy: 0.6979
Epoch 82/100
96/96 - 17s - loss: 0.3995 - accuracy: 0.9336 - val_loss: 4.7204 - val_accuracy: 0.6116
Epoch 83/100
96/96 - 17s - loss: 1.0538 - accuracy: 0.8792 - val_loss: 5.1493 - val_accuracy: 0.6132
Epoch 84/100
96/96 - 17s - loss: 0.4331 - accuracy: 0.9370 - val_loss: 4.8566 - val_accuracy: 0.6580
Epoch 85/100
96/96 - 17s - loss: 0.2731 - accuracy: 0.9665 - val_loss: 4.8847 - val_accuracy: 0.6775
Epoch 86/100
96/96 - 17s - loss: 0.0874 - accuracy: 0.9844 - val_loss: 4.4004 - val_accuracy: 0.7028
Epoch 87/100
96/96 - 17s - loss: 0.0490 - accuracy: 0.9902 - val_loss: 4.4966 - val_accuracy: 0.7109
Epoch 88/100
96/96 - 17s - loss: 0.0186 - accuracy: 0.9969 - val_loss: 4.3135 - val_accuracy: 0.7288
Epoch 89/100
96/96 - 17s - loss: 0.0109 - accuracy: 0.9971 - val_loss: 4.4104 - val_accuracy: 0.7182
Epoch 90/100
96/96 - 17s - loss: 0.0034 - accuracy: 0.9989 - val_loss: 4.5540 - val_accuracy: 0.7256
Epo

In [43]:
import winsound

winsound.Beep(1000, 1000)

# ResNet 50

In [ ]:
from tensorflow.keras.applications  import ResNet50

In [ ]:
resnet = ResNet50(
    input_shape=(224,224,3),
    weights="imagenet",
    include_top=False, # removes last layer of the resnet model
)

In [ ]:
for layer in resnet.layers: #don't train existing weights
    layer.trainable = False

In [ ]:
x = Flatten()(resnet.output)

prediction = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=resnet.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [ ]:
batch_size = 64
epochs = 100
verbose= 2

In [ ]:
EXPERIMENT_NAME = "transfer-learning-tests"
RUN_NAME = "resnet_50"

mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.set_tracking_uri('file:///E:/GoogleSync/Masters/Dissertation/MLflow/mlruns')

with mlflow.start_run(run_name=RUN_NAME) as run:
        
        mlflow.tensorflow.autolog()
        
        history = model.fit(image_train, label_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(image_val, label_val), callbacks=[tensorboard_callback])

        score = model.evaluate(image_test, label_test, batch_size=batch_size, verbose = verbose)
        
        
        
    
        
        mlflow.log_metric("test loss", score[0])
        mlflow.log_metric("test accuracy", score[1])
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)
        

In [ ]:
import winsound

winsound.Beep(1000, 1000)

# ResNet 101

In [ ]:
from tensorflow.keras.applications  import ResNet101

In [ ]:
resnet = ResNet101(
    input_shape=(224,224,3),
    weights="imagenet",
    include_top=False, # removes last layer of the resnet model
)

In [ ]:
for layer in resnet.layers: #don't train existing weights
    layer.trainable = False

In [ ]:
x = Flatten()(resnet.output)

prediction = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=resnet.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [ ]:
batch_size = 64
epochs = 100
verbose= 2

In [ ]:
EXPERIMENT_NAME = "transfer-learning-tests"
RUN_NAME = "resnet_101"

mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.set_tracking_uri('file:///E:/GoogleSync/Masters/Dissertation/MLflow/mlruns')

with mlflow.start_run(run_name=RUN_NAME) as run:
        
        mlflow.tensorflow.autolog()
        
        history = model.fit(image_train, label_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(image_val, label_val), callbacks=[tensorboard_callback])

        score = model.evaluate(image_test, label_test, batch_size=batch_size, verbose = verbose)
        
        
        
    
        
        mlflow.log_metric("test loss", score[0])
        mlflow.log_metric("test accuracy", score[1])
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)
        

In [ ]:
import winsound

winsound.Beep(1000, 1000)

# Experiment InceptionV3

In [ ]:
model = tf.keras.applications.InceptionV3(input_shape=(224,224,3),
    weights="imagenet",
    include_top=False,)

In [ ]:
for layer in model.layers: #don't train existing weights
    layer.trainable = False

In [ ]:
x = Flatten()(model.output)

prediction = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=model.input, outputs=prediction)

In [ ]:
print(model.summary())

In [ ]:
# base_inputs = model.layers[0].input
# base_outputs = model.layers[-2].output
# final_outputs = layers.Dense(102)(base_outputs)
# model = keras.Model(inputs=base_inputs, outputs=final_outputs)


In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="Adam",
    metrics=["accuracy"]
)

In [ ]:
print(model.summary())

In [ ]:
EXPERIMENT_NAME = "transfer-learning-tests"
RUN_NAME = "inception_v3"

mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.set_tracking_uri('file:///E:/GoogleSync/Masters/Dissertation/MLflow/mlruns')

with mlflow.start_run(run_name=RUN_NAME) as run:
        
        mlflow.tensorflow.autolog()
        
        history = model.fit(image_train, label_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(image_val, label_val), callbacks=[tensorboard_callback])

        score = model.evaluate(image_test, label_test, batch_size=batch_size, verbose = verbose)
        
        
        
    
        
        mlflow.log_metric("test loss", score[0])
        mlflow.log_metric("test accuracy", score[1])
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)
        

# VGG 16

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224,224,3))

In [ ]:
base_model.trainable = False

In [ ]:
base_model.summary()

In [ ]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(NUM_CLASSES, activation='softmax')

In [ ]:
model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [ ]:
batch_size = 64
epochs = 100
verbose= 2

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [ ]:
EXPERIMENT_NAME = "transfer-learning-tests"
RUN_NAME = "vgg_16"

mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.set_tracking_uri('file:///E:/GoogleSync/Masters/Dissertation/MLflow/mlruns')

with mlflow.start_run(run_name=RUN_NAME) as run:
        
        mlflow.tensorflow.autolog()
        
        history = model.fit(image_train, label_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(image_val, label_val), callbacks=[tensorboard_callback])

        score = model.evaluate(image_test, label_test, batch_size=batch_size, verbose = verbose)
        
        
        
    
        
        mlflow.log_metric("test loss", score[0])
        mlflow.log_metric("test accuracy", score[1])
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)
        

# VGG 19 

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19

In [ ]:
base_model = VGG19(weights="imagenet", include_top=False, input_shape=(224,224,3))

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
base_model.summary()

In [ ]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(100, activation='relu')
dense_layer_2 = layers.Dense(100, activation='relu')
prediction_layer = layers.Dense(NUM_CLASSES, activation='softmax')

In [ ]:
model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [ ]:
batch_size = 64
epochs = 100
verbose= 2

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [ ]:
EXPERIMENT_NAME = "transfer-learning-tests"
RUN_NAME = "vgg_19"

mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.set_tracking_uri('file:///E:/GoogleSync/Masters/Dissertation/MLflow/mlruns')

with mlflow.start_run(run_name=RUN_NAME) as run:
        
        mlflow.tensorflow.autolog()
        
        history = model.fit(image_train, label_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(image_val, label_val), callbacks=[tensorboard_callback])

        score = model.evaluate(image_test, label_test, batch_size=batch_size, verbose = verbose)
        
        
        
    
        
        mlflow.log_metric("test loss", score[0])
        mlflow.log_metric("test accuracy", score[1])
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)

In [ ]:
batch_size = 64
epochs = 100
verbose= 2

# Xception

In [ ]:
base_model = tf.keras.applications.xception.Xception(
    include_top=False, weights='imagenet', 
    input_shape=(224,224,3),
    classifier_activation='softmax'
)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(100, activation='relu')
dense_layer_2 = layers.Dense(100, activation='relu')
prediction_layer = layers.Dense(NUM_CLASSES, activation='softmax')

In [ ]:
model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [ ]:
EXPERIMENT_NAME = "transfer-learning-tests"
RUN_NAME = "xception"

mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.set_tracking_uri('file:///E:/GoogleSync/Masters/Dissertation/MLflow/mlruns')

with mlflow.start_run(run_name=RUN_NAME) as run:
        
        mlflow.tensorflow.autolog()
        
        history = model.fit(image_train, label_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(image_val, label_val), callbacks=[tensorboard_callback])

        score = model.evaluate(image_test, label_test, batch_size=batch_size, verbose = verbose)
        
        
        
    
        
        mlflow.log_metric("test loss", score[0])
        mlflow.log_metric("test accuracy", score[1])
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)